# Instructions

- Copy all loki result files into **Loki_Files** folder
- Only Loki result should exist in the **Loki_Files** folder
- Result files must be generated by loki with option --csv 




In [30]:
import glob
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [31]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [32]:
files = glob.glob(".\Loki_Files\*")
lines = []
for f in files:
    with open(f, encoding='utf-8') as fh:
        for line in fh:
            lines.append(line)


## Number of files

In [33]:
len(files)

4

In [34]:
cols = ['Hostname','Type', 'File',"Rule Name","Description"]
df = pd.DataFrame(columns = cols)
row = {}
pd.set_option('display.max_colwidth', None)

for line in lines:
    try:
        row['Hostname'] = line.split(',')[1]
    except IndexError:
        continue
        
    if "ALERT" in line or "WARNING" in line:
        if "ALERT" in line:
            row['Type'] = "Alert"  
        elif "WARNING" in line:
            row['Type'] = "Warning"
    else:
        row['Type'] = np.NAN
    match = re.search('FILE:\s(.*?)\sSCORE',line)
    if match:
        row['File'] = re.search('FILE:\s(.*?)\sSCORE',line).group(1)
            
    match = re.search('Yara Rule MATCH:\s(.*?)\s',line)   
    if match:
        row['Rule Name']= re.search('Yara Rule MATCH:\s(.*?)\s',line).group(1)
    match = re.search('DESCRIPTION:\s(.*?):',line)
    if match:
        row['Description']= re.search('DESCRIPTION:\s(.*?)REF:',line).group(1)
      
    df=df.append(row, ignore_index=True)
    


## Number of systems


In [35]:
df['Hostname'].nunique()

4

## Number of systems with alerts or warnings

In [36]:
filt = (df['Type'] == 'Alert' ) | (df['Type'] == 'Warning')
idf=df[filt]
idf['Hostname'].nunique()

3

# Number of warning and alert 

In [37]:
idf["Type"].value_counts()

Warning    11
Alert       7
Name: Type, dtype: int64

## Rule Names 

In [38]:
idf['Rule Name'].value_counts()

psexec_generic                          4
S179_putty                              3
Hacktool_Strings_p0wnedShell            3
S149_WinPcap                            2
APT10_Malware_Sample_Gen                2
FVEY_ShadowBroker_Auct_Dez16_Strings    2
EquationDrug_HDDSSD_Op_RID2F2E          2
Name: Rule Name, dtype: int64

# Top Infected Systems

In [39]:
idf['Hostname'].value_counts()

DS1        10
Server1     4
DS-SER1     4
Name: Hostname, dtype: int64

# Top matching files

In [40]:
idf['File'].value_counts().head(20)

C:\ProgramData\collectt\settings.db                                                         2
C:\SandBoxDeployementScripts\PSTools\PsExec.exe                                             1
C:\Users\sa\Documents\fx1.exe                                                               1
C:\Users\sa\report2.txt                                                                     1
C:\Users\sa\configuration                                                                   1
C:\Users\Administrator\Downloads\pwer.cs                                                    1
C:\Users\Administrator\Downloads\fx1.exe                                                    1
C:\Users\Administrator\tools\fastrack                                                       1
C:\Program Files (x86)\WinPcap\Uninstall.exe                                                1
C:\SandBoxDeployementScripts\PSTools\PsExec64.exe                                           1
C:\Users\sersea\.m2\repository\org\apache\struts\struts-core

In [41]:
idf.groupby(["Hostname", "Rule Name"]).size()

Hostname  Rule Name                           
DS-SER1   S149_WinPcap                            1
          S179_putty                              3
DS1       APT10_Malware_Sample_Gen                2
          EquationDrug_HDDSSD_Op_RID2F2E          2
          FVEY_ShadowBroker_Auct_Dez16_Strings    2
          Hacktool_Strings_p0wnedShell            3
          S149_WinPcap                            1
Server1   psexec_generic                          4
dtype: int64

## All Alerts

In [42]:
idf[idf['Type'] =='Alert'].head(20)

,Hostname,Type,File,Rule Name,Description
41,DS-SER1,Alert,C:\Users\sersea\.m2\repository\org\apache\struts\struts-core\1.3.8\struts-core-1.3.8.jar,S179_putty,Putty
88,DS1,Alert,C:\Users\Administrator\tools\fastrack,FVEY_ShadowBroker_Auct_Dez16_Strings,String from the ShodowBroker Files Screenshots - Dec 2016
89,DS1,Alert,C:\Users\Administrator\Downloads\fx1.exe,EquationDrug_HDDSSD_Op_RID2F2E,EquationDrug - HDD/SSD firmware operation - nls_933w.dll
90,DS1,Alert,C:\Users\Administrator\Downloads\pwer.cs,Hacktool_Strings_p0wnedShell,p0wnedShell Runspace Post Exploitation Toolkit - file p0wnedShell.cs
93,DS1,Alert,C:\Users\sa\report2.txt,FVEY_ShadowBroker_Auct_Dez16_Strings,String from the ShodowBroker Files Screenshots - Dec 2016
94,DS1,Alert,C:\Users\sa\Documents\fx1.exe,EquationDrug_HDDSSD_Op_RID2F2E,EquationDrug - HDD/SSD firmware operation - nls_933w.dll
95,DS1,Alert,C:\Users\sa\Documents\pwer.cs,Hacktool_Strings_p0wnedShell,p0wnedShell Runspace Post Exploitation Toolkit - file p0wnedShell.cs


## All Warnings

In [43]:
idf[idf['Type']=="Warning"].head(20)

,Hostname,Type,File,Rule Name,Description
14,Server1,Warning,C:\SandBoxDeployementScripts\PSTools\PsExec.exe,psexec_generic,not set
15,Server1,Warning,C:\SandBoxDeployementScripts\PSTools\PsExec64.exe,psexec_generic,not set
17,Server1,Warning,C:\Users\admin\Downloads\PSTools\PsExec.exe,psexec_generic,not set
18,Server1,Warning,C:\Users\admin\Downloads\PSTools\PsExec64.exe,psexec_generic,not set
38,DS-SER1,Warning,C:\Program Files\WinPcap\Uninstall.exe,S149_WinPcap,WinPcap
39,DS-SER1,Warning,C:\Program Files (x86)\WinSCP\PuTTY\pageant.exe,S179_putty,Putty
40,DS-SER1,Warning,C:\Program Files (x86)\WinSCP\PuTTY\puttygen.exe,S179_putty,Putty
84,DS1,Warning,C:\Program Files (x86)\WinPcap\Uninstall.exe,S149_WinPcap,WinPcap
85,DS1,Warning,C:\ProgramData\collectt\settings.db,Hacktool_Strings_p0wnedShell,p0wnedShell Runspace Post Exploitation Toolkit - file p0wnedShell.cs
87,DS1,Warning,C:\ProgramData\collectt\settings.db,APT10_Malware_Sample_Gen,APT 10 / Cloud Hopper malware campaign
